In [0]:
#from fuzzywuzzy import fuzz 
#from fuzzywuzzy import process 
#import nltk
from pyspark.sql.functions import *
#from nltk.corpus import stopwords
import pandas as pd
#from nltk.tokenize import sent_tokenize
#from nltk.tokenize import word_tokenize
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import linear_kernel
import re
#from keras.preprocessing.text import Tokenizer
#import tensorflow as tf

In [0]:
# retrieve Chats
df_chats= spark.table("live_admin_chat.chatmessages")
df_chats.show()

+-------------+------+--------------------+--------------------+---------------+-------------+--------------------+
ChatMessageId|ChatId| MessageText| TimeStamp| MessageBy|created_on_s3| sync_time|
+-------------+------+--------------------+--------------------+---------------+-------------+--------------------+
 3401698| 40862|Hi, welcome to th...|12/30/2020 8:00:1...| Brad| 2020-12-31|2021-02-09 12:15:...|
 3401763| 40862| Hey|12/30/2020 8:00:1...| Visitor1595679| 2020-12-31|2021-02-09 12:15:...|
 3401824| 40862| Hi there.|12/30/2020 8:00:2...| Brad| 2020-12-31|2021-02-09 12:15:...|
 3401831| 40862| this is the 3rd msg|12/30/2020 8:00:2...| Visitor1595679| 2020-12-31|2021-02-09 12:15:...|
 3401957| 40862|Let me help you w...|12/30/2020 8:00:3...| Brad| 2020-12-31|2021-02-09 12:15:...|
 3402265| 40862|i had an issue wi...|12/30/2020 8:01:1...| Visitor1595679| 2020-12-31|2021-02-09 12:15:...|
 3402697| 40862|Let me check it f...|12/30/2020 8:01:5...| Brad| 2020-12-31|2021-02-09 12:15:...|
 3402818| 40862|[NAVIGATEURL] Vis...|12/30/2020 8:02:1...| Visitor5099| 2020-12-31|2021-02-09 12:15:...|
 3403213| 40862|I can see from my...|12/30/2020 8:02:5...| Brad| 2020-12-31|2021-02-09 12:15:...|
 3404685| 40862|Can u plz call me...|12/30/2020 8:05:4...| Visitor1595679| 2020-12-31|2021-02-09 12:15:...|
 3404870| 40862|I’m struggling to...|12/30/2020 8:06:0...| Visitor1595679| 2020-12-31|2021-02-09 12:15:...|
 3405297| 40862|I'm sorry but we ...|12/30/2020 8:06:5...| Brad| 2020-12-31|2021-02-09 12:15:...|
 3405611| 40862|Can you please co...|12/30/2020 8:07:3...| Brad| 2020-12-31|2021-02-09 12:15:...|
 3401946| 40866|Hi, welcome to th...|12/30/2020 8:00:3...| Harry| 2020-12-31|2021-02-09 12:15:...|
 3401958| 40866| Hiiii|12/30/2020 8:00:4...|Osama Abuayhour| 2020-12-31|2021-02-09 12:15:...|
 3402021| 40866|Hello there! how ...|12/30/2020 8:00:4...| Harry| 2020-12-31|2021-02-09 12:15:...|
 3402708| 40866|Are we still conn...|12/30/2020 8:02:0...| Harry| 2020-12-31|2021-02-09 12:15:...|
 3402880| 40866|Just I want to do...|12/30/2020 8:02:2...|Osama Abuayhour| 2020-12-31|2021-02-09 12:15:...|
 3403068| 40866|I have your email...|12/30/2020 8:02:4...| Harry| 2020-12-31|2021-02-09 12:15:...|
 3403256| 40866|I contacted you a...|12/30/2020 8:03:0...|Osama Abuayhour| 2020-12-31|2021-02-09 12:15:...|
+-------------+------+--------------------+--------------------+---------------+-------------+--------------------+
only showing top 20 rows

In [0]:
display(df_chats.filter((df_chats.ChatId==54137)))

ChatMessageId,ChatId,MessageText,TimeStamp,MessageBy,created_on_s3,sync_time
5380468,54137,"Hi, welcome to the ENTERTAINER Live Chat. How may I help you?",1/2/2021 10:23:37 AM,Jim,2021-01-02,2021-02-09T12:16:35.095+0000
5380483,54137,Hi,1/2/2021 10:23:38 AM,Visitor2296144,2021-01-02,2021-02-09T12:16:35.095+0000
5380658,54137,I spoke to Stella earlier regarding my recent Gourmet 2021 purchase,1/2/2021 10:23:55 AM,Visitor2296144,2021-01-02,2021-02-09T12:16:35.095+0000
5381088,54137,Hi! How may I help you?,1/2/2021 10:24:44 AM,Jim,2021-01-02,2021-02-09T12:16:35.095+0000
5381212,54137,I want to change my purchase from Gourmet 2021 to Classic. Some of the restaurants from Gourmet 2020 is not in 2021 anymore and the listed restaurants are very less now.,1/2/2021 10:24:56 AM,Visitor2296144,2021-01-02,2021-02-09T12:16:35.095+0000
5381646,54137,Exchange of product is not possible.,1/2/2021 10:25:45 AM,Jim,2021-01-02,2021-02-09T12:16:35.095+0000
5382140,54137,[NAVIGATEURL] Visitor browsing : https://hub.theentertainerme.com/article-categories/faqs?utm_campaign=settings-help%26utm_medium=link%26header=no%26utm_source=app,1/2/2021 10:26:42 AM,Visitor7793-2296144CID54137,2021-01-02,2021-02-09T12:16:35.095+0000
5382208,54137,Okay I want refund,1/2/2021 10:26:51 AM,Visitor2296144,2021-01-02,2021-02-09T12:16:35.095+0000
5382820,54137,Gourmet 2021 doesn’t have much restaurants now compared to 2020 and the restaurants that I like is not there anymore,1/2/2021 10:28:01 AM,Visitor2296144,2021-01-02,2021-02-09T12:16:35.095+0000
5383197,54137,[NAVIGATEURL] Visitor browsing : https://hub.theentertainerme.com/article-categories/faqs?utm_campaign=settings-help%26utm_medium=link%26header=no%26utm_source=app,1/2/2021 10:28:46 AM,Visitor7793-2296144CID54137,2021-01-02,2021-02-09T12:16:35.095+0000


In [0]:
display(df_chats.filter((df_chats.MessageBy=='Visitor2132837')|(df_chats.MessageBy=='Cheryl')| (df_chats.ChatId==51072)))

In [0]:
df_chats_pd=df_chats.toPandas()
df_chats_pd.head()

Out[4]:

,ChatMessageId,ChatId,MessageText,TimeStamp,MessageBy,created_on_s3,sync_time
0,4924497,51072,"Hi, welcome to the ENTERTAINER Live Chat. How ...",1/1/2021 8:00:56 PM,Cheryl,2021-01-02,2021-02-09 12:16:35.095542
1,4924514,51072,Hi,1/1/2021 8:00:56 PM,Visitor2132837,2021-01-02,2021-02-09 12:16:35.095542
2,4924626,51072,Shaik Ahmed Nawaz,1/1/2021 8:01:06 PM,Visitor2132837,2021-01-02,2021-02-09 12:16:35.095542
3,4924686,51072,[NICKCHANGE],1/1/2021 8:01:11 PM,Shaik Ahmed Nawaz,2021-01-02,2021-02-09 12:16:35.095542
4,4924689,51072,Sanawaz2k@gmail.com,1/1/2021 8:01:12 PM,Visitor2132837,2021-01-02,2021-02-09 12:16:35.095542


In [0]:
new_df=df_chats_pd[["ChatId","MessageText","MessageBy"]].groupby(["ChatId","MessageBy"],as_index=False).aggregate(lambda x: list(x))
new_df.head()

Out[5]:

,ChatId,MessageBy,MessageText
0,13003,Chadli Khaled,"[[NICKCHANGE], What do you means by up to merc..."
1,13003,Stella,"[Hi, welcome to the ENTERTAINER Live Chat. How..."
2,13003,Visitor11251,[Hi! I’m at Liwan restaurant for the Friday br...
3,13003,Visitor5363-11251CID13003,[[NAVIGATEURL] Visitor browsing : https://hub....
4,13008,7466,[[NAVIGATEURL] Please reply here.]


In [0]:
new_df.loc[new_df['ChatId'] == 54137].head(50)

Out[15]:

,ChatId,MessageBy,MessageText
34694,54137,2637,[[NAVIGATEURL] Please reply here.]
34695,54137,Jim,"[Hi, welcome to the ENTERTAINER Live Chat. How..."
34696,54137,Sandrhea Sacares,"[[NICKCHANGE], Sure, Yes,I did. Disappointed a..."
34697,54137,Visitor2296144,"[Hi, I spoke to Stella earlier regarding my re..."
34698,54137,Visitor7793-2296144CID54137,[[NAVIGATEURL] Visitor browsing : https://hub....


In [0]:
new_df['messages_split']=new_df['MessageText'].apply(lambda x: " ".join(a for a in x))
new_df.head()

Out[17]:

,ChatId,MessageBy,MessageText,messages_split
0,13003,Chadli Khaled,"[[NICKCHANGE], What do you means by up to merc...",[NICKCHANGE] What do you means by up to mercha...
1,13003,Stella,"[Hi, welcome to the ENTERTAINER Live Chat. How...","Hi, welcome to the ENTERTAINER Live Chat. How ..."
2,13003,Visitor11251,[Hi! I’m at Liwan restaurant for the Friday br...,Hi! I’m at Liwan restaurant for the Friday bru...
3,13003,Visitor5363-11251CID13003,[[NAVIGATEURL] Visitor browsing : https://hub....,[NAVIGATEURL] Visitor browsing : https://hub.t...
4,13008,7466,[[NAVIGATEURL] Please reply here.],[NAVIGATEURL] Please reply here.


In [0]:
new_df=new_df[~(new_df.messages_split.str.contains('Hi, welcome to the ENTERTAINER Live Chat'))]
new_df=new_df[~(new_df.messages_split.str.contains('Your patience is much appreciated'))]
new_df=new_df[~(new_df.messages_split.str.contains('NAVIGATEURL'))]
new_df.head()

Out[18]:

,ChatId,MessageBy,MessageText,messages_split
0,13003,Chadli Khaled,"[[NICKCHANGE], What do you means by up to merc...",[NICKCHANGE] What do you means by up to mercha...
2,13003,Visitor11251,[Hi! I’m at Liwan restaurant for the Friday br...,Hi! I’m at Liwan restaurant for the Friday bru...
6,13008,Remon Makrm,"[[NICKCHANGE], Remon_makrm@yahoo.com, The orde...",[NICKCHANGE] Remon_makrm@yahoo.com The order c...
7,13008,Visitor11715,"[Hi, I made order before 30 miutes, Remon makrm]",Hi I made order before 30 miutes Remon makrm
10,13011,Visitor11957,"[Is today a black out day, Ok]",Is today a black out day Ok


In [0]:
new_df.loc[new_df['ChatId'] == 54137].head(50)

Out[19]:

,ChatId,MessageBy,MessageText,messages_split
34696,54137,Sandrhea Sacares,"[[NICKCHANGE], Sure, Yes,I did. Disappointed a...","[NICKCHANGE] Sure Yes,I did. Disappointed as t..."
34697,54137,Visitor2296144,"[Hi, I spoke to Stella earlier regarding my re...",Hi I spoke to Stella earlier regarding my rece...


In [0]:
new_df.loc[new_df['ChatId'] == 54137].messages_split.values

Out[21]: array(['[NICKCHANGE] Sure Yes,I did. Disappointed as the restaurants listed in Gourmet 2020 is not there in 2021 anymore Yes. How long will it take to reflect in the bank? Can I know why some of the restaurants from Gourmet 2020 is now available in Classic 2021? No worries. I can wait. I just don’t think the Gourmet 2021 is a good value for money anymore 0565654631',
 'Hi I spoke to Stella earlier regarding my recent Gourmet 2021 purchase I want to change my purchase from Gourmet 2021 to Classic. Some of the restaurants from Gourmet 2020 is not in 2021 anymore and the listed restaurants are very less now. Okay I want refund Gourmet 2021 doesn’t have much restaurants now compared to 2020 and the restaurants that I like is not there anymore sandrheasacares@gmail.com Sandrhea Sacares'],
 dtype=object)

In [0]:
def prep_chat_data():
  
  df_chats= spark.table("live_admin_chat.chatmessages")
  
  df_chats_pd=df_chats.toPandas()
  
  new_df=df_chats_pd[["ChatId","MessageText","MessageBy"]].groupby(["ChatId","MessageBy"],as_index=False).aggregate(lambda x: list(x))
  new_df['MessageText']=new_df['MessageText'].apply(lambda x: " ".join(a for a in x))
  new_df=new_df[~(new_df.MessageText.str.contains('Hi, welcome to the ENTERTAINER Live Chat'))]
  new_df=new_df[~(new_df.MessageText.str.contains('Your patience is much appreciated'))]
  new_df=new_df[~(new_df.MessageText.str.contains('NAVIGATEURL'))]
  
  new_df_sp = spark.createDataFrame(new_df)
  new_df_sp = new_df_sp.select('ChatId','MessageText').groupby('ChatId').agg(collect_set('MessageText')).sort(['ChatId'],ascending=True)
  
  new_df_pd=new_df_sp.toPandas()
  new_df_pd.rename(columns={"collect_set(MessageText)":"MessageText"},inplace=True)
  new_df_pd['MessageText']=new_df_pd['MessageText'].apply(lambda x: " ".join(a for a in x))
  
  new_df_pd['MessageText'] = new_df_pd['MessageText'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  
  freq=['nickchange','[nickchange]','u','ok','okay','hi','thx','hello','entertainer','thanks','thankyou','thank you']
  new_df_pd['MessageText'] = new_df_pd['MessageText'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
  new_df.reset_index(drop=True,inplace = True)
  
  df_chats=spark.createDataFrame(new_df_pd)
  df_chats=df_chats.select('ChatId','MessageText')
  return df_chats

In [0]:
chats=prep_chat_data()

In [0]:
display(chats)

ChatId,MessageText
13003,"hi! i’m at liwan restaurant for the friday brunch and i would like to know if the offer of but one get one free applies for kids as well chadli khaled chadli.khaled@yahoo.fr what do you means by up to merchant discretion if i purchase 1 , i’m supposed to get 1 free , for adults or for kids i’m a bit lost"
13008,remon_makrm@yahoo.com the order cancled and the amount debited of my account i call restruant tolde not receive my orddr and i reoorder again my issue the mount debited two times amount but i received sms from bank this amonut already dudected ! i will check with my bank thnks for your help i made order before 30 miutes remon makrm
13011,is today a black out day
13012,"hey there! what can i do for you today? i am afraid, you cannot use vouchers today. as it is one of our exclusion days. https://www.theentertainerme.com/rules-of-use?v=full please view the link above. view the ""exclusion days"" tab please. it's a public holiday. would you like my assistance with anything else? i want to take my kids to the ice rink and want to use the public session buy 1 get 1 free i can use it today? coz last time they told me not possible why? why its exclusion day"
13016,"also all gourmet is not showing although i have it even in 2020 not all i have 2021 classic only gourmet is not shown i have purchased kitchen 6 coupons also from the points now is not showing no, do i have to do this no will try i cannot see 2021 offers i have bought uae bundle samar abushaban samargawdat@gmail.com"
13022,the cant be used on friday? why its mentioned 7 days a week
13027,i was told by harry’s harbourfront centre that they do not give offers today but this app never mentioned anything about public holidays and such dennis tan tandennis887@gmail.com
13034,"hey there! what can i do for you today? alright, let me check that for you. meanwhile, may i please have your full name and email address so that i'd be able to assist you better? i would suggest you to contact your bank for the confirmation of the refunded amount. it is mentioned in the screen shot you shared with us. it usually takes 10-14 days for the refund however it depends on the bank. yes as per your email the amount has refund has been processed. your welcome. have a good day. i booked for 2 nights with hsbc using the buy 1 get 1 free with swissotel hotel however i got the attached message and the hotel refused that i stay with them at that rate as also they never received my booking from my end . can you ensure the aed 1621 is refunded. http://files.thelivechatsoftware.com/fd1fa432_f096_485d_818d_05f8af9c0c60e5yoyoz_2(802.28kb).png when was it refunded? can you confirm that it was refunded from your side, with date if refund and reference number so the anount had been refunded right? amount thxs"
13035,"وعليكم السلام ورحمة الله وبركاته مرحبا, كيف يمكنني مساعدتك ؟ قبل ان نبدأ، هل لي بمعرفة اسمك الكامل وعنوان بريدك الإلكتروني من فضلك؟ هذا سيساعد في متابعة استفسارك ومساعدتك بشكل أفضل شكراً لمشاركة البريد الإلكتروني الخاص بك معنا. من فضلك، هل لي أيضا أن أعرف سبب رفض العرض؟ هل لي بمعرفة المكان أو الفرع الذي حدثت فيه المشكلة؟ لا ليس شرطاً يمكنك استخدام 4 كوبونات في نفس الطاولة بشرط حضور 4 اشخاص الرجاء ذكر الفروع التي حصلت فيها المشاكل متى كانت زيارتك للتاجر؟ هل لي بمعرفة مع من تكلمت؟ شكراً لك على هذة المعلومات سأقوم بتحويلك لأحد ممثلي الخدمة لدينا ليقوم بمعاودة التواصل معك بخصوص هذه المشكلة هل لي برقم هاتفك مسبوقاً برمز الدولة لسجلاتنا من فضلك؟ شكرًا على مشاركتك ذلك معنا. سيتواصل معك أحد ممثلي خدمة العملاء لدينا خلال 48-72 ساعة عمل هل هناك شيء آخر يمكنني مساعدتك به ؟ شكرا جزيلا لزيارتك موقع انترتينر, نتمنى أن تنال خدماتنا إعجابكم! هلا فيك 00966570591866 شكرا عفوا السلام عليكم لو سمحت دلوقتى اكتر من مطعم رافض استخدام انترتنر او بيشترط وجبات محددة او استخدام كوبون واحد دعاء m.sayyid.othman@gmail.com مثلا مطعم تامو كرامبل تشكس كلمارى دول من الاساس رافضين انترتنر مطعم عنجر اشترط وجبات محددة وكمان لازم يتاكد ان مجموع استخدامنا تلات كوبونات يعنى لو بدلت كوبون ليه لا مش هياخده

In [0]:
dbutils.library.installPyPI("pycld2")
# from googletrans import Translator

Out[59]: True

In [0]:
chats_pd=chats.toPandas()
chats_pd.head()
#chats['language']

Out[66]:

,ChatId,MessageText
0,13003,hi! i’m at liwan restaurant for the friday bru...
1,13008,remon_makrm@yahoo.com the order cancled and th...
2,13011,is today a black out day
3,13012,hey there! what can i do for you today? i am a...
4,13016,also all gourmet is not showing although i hav...


In [0]:
def detectect_land(text):
    #print(text)
    try:
      _, _, _, detected_language = cld2.detect(text,  returnVectors=True)
      return detected_language[0][2]
    except:
      #print('detected_language',detected_language)
      return 'UNKNOWN'
    

In [0]:
chats_pd['language']=chats_pd.MessageText.apply(detectect_land)
chats_pd.head()

Out[92]:

,ChatId,MessageText,language
0,13003,hi! i’m at liwan restaurant for the friday bru...,ENGLISH
1,13008,remon_makrm@yahoo.com the order cancled and th...,ENGLISH
2,13011,is today a black out day,ENGLISH
3,13012,hey there! what can i do for you today? i am a...,ENGLISH
4,13016,also all gourmet is not showing although i hav...,ENGLISH


In [0]:
chats_pd=chats_pd[~(chats_pd.language.str.contains('ARABIC'))]
chats_pd=chats_pd[~(chats_pd.language.str.contains('UNKNOWN'))]
chats_pd=chats_pd[~(chats_pd.language.str.contains('ARABIC'))]
chats_pd=chats_pd[~(chats_pd.language.str.contains('Unknown'))]
chats_pd.count()

Out[104]: ChatId 28778
MessageText 28778
language 28778
dtype: int64

In [0]:
final_chats=spark.createDataFrame(chats_pd)
final_chats=final_chats.select('ChatId','MessageText')

In [0]:
display(final_chats)

ChatId,MessageText
13003,"hi! i’m at liwan restaurant for the friday brunch and i would like to know if the offer of but one get one free applies for kids as well chadli khaled chadli.khaled@yahoo.fr what do you means by up to merchant discretion if i purchase 1 , i’m supposed to get 1 free , for adults or for kids i’m a bit lost"
13008,remon_makrm@yahoo.com the order cancled and the amount debited of my account i call restruant tolde not receive my orddr and i reoorder again my issue the mount debited two times amount but i received sms from bank this amonut already dudected ! i will check with my bank thnks for your help i made order before 30 miutes remon makrm
13011,is today a black out day
13012,"hey there! what can i do for you today? i am afraid, you cannot use vouchers today. as it is one of our exclusion days. https://www.theentertainerme.com/rules-of-use?v=full please view the link above. view the ""exclusion days"" tab please. it's a public holiday. would you like my assistance with anything else? i want to take my kids to the ice rink and want to use the public session buy 1 get 1 free i can use it today? coz last time they told me not possible why? why its exclusion day"
13016,"also all gourmet is not showing although i have it even in 2020 not all i have 2021 classic only gourmet is not shown i have purchased kitchen 6 coupons also from the points now is not showing no, do i have to do this no will try i cannot see 2021 offers i have bought uae bundle samar abushaban samargawdat@gmail.com"
13022,the cant be used on friday? why its mentioned 7 days a week
13027,i was told by harry’s harbourfront centre that they do not give offers today but this app never mentioned anything about public holidays and such dennis tan tandennis887@gmail.com
13034,"hey there! what can i do for you today? alright, let me check that for you. meanwhile, may i please have your full name and email address so that i'd be able to assist you better? i would suggest you to contact your bank for the confirmation of the refunded amount. it is mentioned in the screen shot you shared with us. it usually takes 10-14 days for the refund however it depends on the bank. yes as per your email the amount has refund has been processed. your welcome. have a good day. i booked for 2 nights with hsbc using the buy 1 get 1 free with swissotel hotel however i got the attached message and the hotel refused that i stay with them at that rate as also they never received my booking from my end . can you ensure the aed 1621 is refunded. http://files.thelivechatsoftware.com/fd1fa432_f096_485d_818d_05f8af9c0c60e5yoyoz_2(802.28kb).png when was it refunded? can you confirm that it was refunded from your side, with date if refund and reference number so the anount had been refunded right? amount thxs"
13037,"i had kitchen 6 jw marriot yes, and earlier 2020 dubai classic and gourmet exactly, it has been in 2020 and now is not visible i bought the vouchers by 2020 points and planned to go this week what is the status now now i used 1000 points, you will return back? 0503049079 i have purchased coupons in kitchen 6 restuar now the restaurant is not showing samar abu shaban samargawdat@gmail.com"
13040,"i i bought 2021 offers .. so i will get 2020 offers also if ?? but i didn’t buy 2020 offers before drsara_drsara@hotmail.com sara sakr how no still i was asking first another question so i will not get 2020 offer?? for free as you said and what about today can i use these offers for today?? in kuwait yes you will get 2020 free. meanwhile, may i please have your full name and email address so that i'd be able to assist you better? thank you for sharing the details with me. let me guide you with it. i can't see any product against your email. did you purchase it recently? as we are providing 2020 free with 2021, you will be able to use offers in 2020 till 31st dec 2020. and you'll be able to use offers of 2021 from 2nd jan 2021 till 31st dec 2021. yes go on. you will get 2020

In [0]:
chats_pd.loc[chats_pd['language'] == 'Unknown'].head(50)

Out[103]:

,ChatId,MessageText,language
15,13055,does work today,Unknown
27,13092,[transferchat]_transferred by brady to ali (عر...,Unknown
79,13324,anas maktabi maktabi500@yahoo.com order number...,Unknown
107,13435,[transferchat]_transferred by sophia to ali (ع...,Unknown
115,13467,[transferchat]_transferred by sophia to ali (ع...,Unknown
131,13532,[transferchat]_transferred by sophia to ali (ع...,Unknown
143,13595,[transferchat]_transferred by sophia to haroon...,Unknown
153,13643,swati deorah dittytextile@gmail.com ski dubai ...,Unknown
172,13793,swati deorah dittytextile@gmail.com speaking t...,Unknown
186,13864,visitor70348 liked this conversation مساء الخي...,Unknown


In [0]:
import pycld2 as cld2
text_content = """[transferchat]_transferred by sophia to ali (عربي (ar)) (la) [qcratedchat] hala rasheed gave rating 3 to ali's chat on theentertainerme.com/en /la اتفضل طيب الان اشوفها اوك جزاك الله خير وعليكم السلام ورحمة الله وبركاته مرحبا, كيف يمكنني مساعدتك ؟ من دواعي سروري وجودك معنا قبل ان نبدأ، هل لي بمعرفة اسمك الكامل وعنوان بريدك الإلكتروني من فضلك؟ هذا سيساعد في متابعة استفسارك ومساعدتك بشكل أفضل شكراً لمشاركة البريد الإلكتروني الخاص بك معنا. نعم ، تم شراء المنتج على هذا الايميل rami_tayeb@hotmail.com والمنتج موجود في حسابك ماهي الرسالة التي تظهر هل يظهر قفل على العروض ؟ عروض 2021 غير متاحة في الوقت الحالي ، سوف تكون متاحة من تاريخ 2 يناير 2021 نعم عروض 2020 مقدمة كهدية وموجودة في حسابك هل تقصد عروض 2020؟ تتم مضاعفة العروض في نهاية العام يتم ذلك وعند الدخول في العام الجديد تكون العروض جديدة العروض صالحة من 2 يناير لغاية 30 ديسمبر من كل عام هل وضحت الفكرة نعم ، عروض 2020 سوف تكون غير متوفرة في تاريخ 30 ديسمبر 2020 اي نهاية هذا الشهر شكرًا لك، هل لديك أي استفسار آخر ؟ عروض 2020 في حالة انت مشت..."""
_, _, _, detected_language = cld2.detect(text_content,  returnVectors=True)
print(detected_language)

((0, 55, 'Unknown', 'un'), (55, 96, 'ENGLISH', 'en'), (151, 601, 'ARABIC', 'ar'), (752, 23, 'Unknown', 'un'), (775, 865, 'ARABIC', 'ar'))

In [0]:
# detected_language[0][2]

In [0]:
chats.count()

Out[40]: 32541

In [0]:
display(chats.filter(chats.ChatId == 18754))

ChatId,MessageText
18754,


In [0]:
chats.loc[chats['ChatId'] == 54137].messages_split.values

Out[28]: array(['[NICKCHANGE] Sure Yes,I did. Disappointed as the restaurants listed in Gourmet 2020 is not there in 2021 anymore Yes. How long will it take to reflect in the bank? Can I know why some of the restaurants from Gourmet 2020 is now available in Classic 2021? No worries. I can wait. I just don’t think the Gourmet 2021 is a good value for money anymore 0565654631',
 'Hi I spoke to Stella earlier regarding my recent Gourmet 2021 purchase I want to change my purchase from Gourmet 2021 to Classic. Some of the restaurants from Gourmet 2020 is not in 2021 anymore and the listed restaurants are very less now. Okay I want refund Gourmet 2021 doesn’t have much restaurants now compared to 2020 and the restaurants that I like is not there anymore sandrheasacares@gmail.com Sandrhea Sacares'],
 dtype=object)